In [1]:
require 'torch'
require 'nn'
require 'optim'
hdf5 = require 'hdf5'

## Hyperparameters
* Filter windows of height $h$ are applied in sizes of $3,4,5$ with TODO feature
maps. Specifically the filters are applied as a matrix $W \in \mathbb{R}^{h \times k}$
to a window of size $h \times k$.
* The stride of the filters is $1$ to iterate over all possible windows of words
(*narrow convolution*).
* A dropout of rate $\rho = .5$ is applied at training time to prevent overfitting.
* An $l_2$ constraint $s = 3$ is applied in (Kim, 2014). However (Zhang and Wallace, 2015)
found that this constraint had little effect on the performance of the network.

In [2]:
-- metadata
epochs = 4
classes = {'1', '2', '3', '4', '5', '6', '7', '8'}
sent_len = 150

## Data
Input format is a matrix $M \in \mathbb{R}^{n} \times k$
with:
* $n$ number of words in the document
* $k$ the dimension of the word vectors obtained from word2vec
* the data is zero-padded to the length of the longest document
(alternatively to a given maximum length). All not-taken word spaces
are set to zero in the matrix.

In [3]:
function get_kth_partition(k)
    fname = "../data/prepped_train_X_word2vec_wordvecs_"..k..".h5"
    print("retrieving "..fname)
    file = hdf5.open(fname, 'r')
    data = file:read():all()
    file:close()

    return data["dataset_"..k.."_x_train"]
        , data["dataset_"..k.."_y_train"]
        , data["dataset_"..k.."_x_test"]
        , data["dataset_"..k.."_y_test"]
end

In [4]:
-- FIXME: replace this with something better
function copy_example(example)
    local X_i = torch.Tensor(6, sent_len, example:size()[2]):zero()

    for j = 1, example:size()[1] do
        X_i[{1, j}] = example[j]
        X_i[{2, j}] = example[j]
        X_i[{3, j}] = example[j]
        X_i[{4, j}] = example[j]
        X_i[{5, j}] = example[j]
        X_i[{6, j}] = example[j]
    end

    return X_i
end

In [5]:
-- binarize label probabilities
function binarize_pred_labels(labels)
    bin = torch.Tensor(labels:size()[1])

    for i = 1, labels:size()[1] do
        if labels[i] > .5 then
            bin[i] = 1
        else
            bin[i] = 0
        end
    end

    return bin
end

## Model

In [6]:
function get_model(sent_len, wordvec_len, dropout_rho, l2_constr)
    local model = nn.Sequential()
    model:add(nn.Identity())

    -- stage 1: convolutions

    -- in: applies the following declared layers to the first dimension of the
    -- tensor (see above.)
    --
    -- out: concatenating the three concatenated max-pooled values to a vector
    -- fed to the fully connected softmax layer yielding the outputs

    -- in: tensor of 6 x num_exaples x sent_len x wordvec_len
    --p = nn.ConcatTable()
    local p = nn.Parallel(1,1)

    -- this is a convolution unit differing in the height of the filter being
    -- applied. each filter is used double time to further improve performance.
    -- each filter yields a feature map, thus for each region size we then
    -- have two feature maps, and in total then six (if using default n-grams
    -- 3 to 5 like (Kim, 2014))
    for i = 3, 5 do
        -- elements of the convolution
        local s1 = nn.Sequential()
        local s2 = nn.Sequential()

        -- input: sent_len x wordvec_len
        s1:add(nn.Reshape(1, sent_len, wordvec_len))
        s2:add(nn.Reshape(1, sent_len, wordvec_len))

        -- takes size of input plane (we only have one channel though)
        -- as well as output plane (again, using only one channel)
        -- kernel width and kernel height as third and fourth arguments

        -- in: 1 x sent_len x wordvec_len
        -- args: input channels, output channels, kernel width, kernel height
        s1:add(nn.SpatialConvolution(1, 1, wordvec_len, i))
        s2:add(nn.SpatialConvolution(1, 1, wordvec_len, i))
        -- out: sent_len - filter_height + 1

        -- non-linearities
        -- in: sent_len - filter_height + 1
        s1:add(nn.ReLU())
        s2:add(nn.ReLU())
        -- out: sent_len - filter_height + 1

        -- the viewed region of the matrix for max-pooling shall be the
        -- size of the matrix, as we want all values to be considered at
        -- once for a single maximum for each filter map.
        -- in: sent_len - filter_height + 1
        s1:add(nn.SpatialMaxPooling(1, sent_len - i + 1))
        s2:add(nn.SpatialMaxPooling(1, sent_len - i + 1))
        -- out: scalar

        p:add(s1)
        p:add(s2)
    end

    model:add(p)

    -- [6x1x1] -> [1x6]
    model:add(nn.View(1,6))

    -- stage 2: fully connected softmax layer
    model:add(nn.Normalize(2, l2_constr))
    model:add(nn.Dropout(dropout_rho))
    model:add(nn.Linear(6, 8))

    --model:add(nn.LogSoftMax()) -- for ClassNLLCriterion
    model:add(nn.Sigmoid())

    return model
end

In [7]:
function train(model, criterion, batch_size, X_k, Y_k)
    local no_examples = X_k:size()[1]
    local parameters, grad_parameters = model:getParameters()
    model:training()
    local tic = os.clock()

    print("training network")

    local next_batch = function(offset, inputs, targets)
        end_offset = offset + batch_size - 1

        -- calculate size of remaining items if we have less than
        -- batch_size examples left.
        if end_offset > inputs:size()[1] then
            end_offset = offset + (inputs:size()[1] - offset)
        end

        print("getting "..offset.." to "..end_offset.." of "..inputs:size()[1])

        return inputs[{{offset, end_offset}, {}, {}}], targets[{{offset, end_offset}, {}}]
    end

    for i = 1, no_examples, batch_size do
        inputs, targets = next_batch(i, X_k, Y_k)
        num_inputs = inputs:size()[1]

        local feval = function(x)
            if x ~= parameters then
                parameters:copy(x)
            end

            grad_parameters:zero()

            local f = 0
            for i = 1, num_inputs do
                local X_k_i = copy_example(inputs[i])
                local output = model:forward(X_k_i):reshape(8)
                local bin_output = binarize_pred_labels(output)
                local targets_i = targets[i]:type("torch.DoubleTensor")

                local err = criterion:forward(output, targets_i)
                f = f + err

                local df_do = criterion:backward(bin_output, targets_i)
                model:backward(inputs[i], df_do)
            end

            grad_parameters:div(num_inputs)
            f = f / num_inputs

            return f, grad_parameters
        end

        optim.adadelta(feval, parameters, {})
    end

    print("done.")
    local toc = os.clock() - tic
    print("took "..toc.."s")
end

In [8]:
function evaluate(model, inputs, targets)
    local c = torch.Tensor(8, 4):zero()

    local preds = {}
    model:evaluate()

    -- check for TN, TP, FN, FP
    for i = 1, inputs:size()[1] do
        local prediction = model:forward(copy_example(inputs[i])):reshape(8)
        preds[#preds + 1] = prediction

        for j = 1, 8 do
            if torch.ceil(prediction[j]) ~= targets[i][j] then
                if prediction[j] >= .5 and targets[i][j] <= .5 then
                    -- false positive
                    c[j][3] = c[j][3] + 1
                else
                    -- false negative
                    c[j][4] = c[j][4] + 1
                end
            else
                if prediction[j] >= .5 then
                    -- true positive
                    c[j][1] = c[j][1] + 1
                else
                    -- true negative
                    c[j][2] = c[j][2] + 1
                end
            end
        end
    end

    return c, preds
end

In [ ]:
confusions = {}
predictions = {}

for k = 1, 10 do
    local criterion = nn.BCECriterion()
    local model = get_model(150, 300, .5, 3)
    local x_train, y_train, x_test, y_test = get_kth_partition(k-1)

    for e = 1, epochs do
        print("training epoch "..e)
        -- train with batch_size = 100
        train(model, criterion, 100, x_train, y_train)
    end

    local c, preds = evaluate(model, x_test, y_test)
    predictions[k] = preds
    confusions[k] = c
end

retrieving ../data/prepped_train_X_word2vec_wordvecs_0.h5	


training epoch 1	
training network	
getting 1 to 100 of 1930	


In [ ]:
-- evaluating confusion "matrices". not real ones but class-wise TP, TN, FP, FN
-- for each of the k partitioned datasets.
-- now we need to calculate macro/micro F1/R/P
local P_micro_aggr = 0
local R_micro_aggr = 0
local P_macro_aggr = 0
local R_macro_aggr = 0

-- the metrics are evaluated for each dataset.
-- the micro averaging is calculated on the cumulative tp/fp/fn
-- for a class while the macro averaging calculates the metric
-- for each class, sums these up and divides them by the class
-- count

for i = 1, #confusions do
    print('set '..i)
    local P_aggr = 0
    local R_aggr = 0
    local TP_aggr = 0
    local TN_aggr = 0
    local FP_aggr = 0
    local FN_aggr = 0

    -- TP, TN, FP, FN
    for j = 1, 8 do
        print('class '..j)
        local TP_i = confusions[i][j][1]
        print('TP '..TP_i)
        local TN_i = confusions[i][j][2]
        print('TN '..TN_i)
        local FP_i = confusions[i][j][3]
        print('FP '..FP_i)
        local FN_i = confusions[i][j][4]
        print('FN '..FN_i)

        TP_aggr = TP_aggr + TP_i
        TN_aggr = TN_aggr + TN_i
        FP_aggr = FP_aggr + FP_i
        FN_aggr = FN_aggr + FN_i

        -- macro
        local P_i = (TP_i + 1) / (TP_i + FP_i + 1)
        P_aggr = P_aggr + P_i

        local R_i = (TP_i + 1) / (TP_i + FN_i + 1)
        R_aggr = R_aggr + R_i
    end

    print('  P macro: '..(P_aggr / 8))
    print('  R macro: '..(R_aggr / 8))
    print('  P micro: '..((TP_aggr + 1) / (TP_aggr + FN_aggr + 1)))
    print('  R micro: '..((TP_aggr + 1) / (TP_aggr + FP_aggr + 1)))

    P_macro_aggr = P_macro_aggr + (P_aggr / 8)
    R_macro_aggr = R_macro_aggr + (R_aggr / 8)
    R_micro_aggr = R_micro_aggr + ((TP_aggr + 1) / (TP_aggr + FP_aggr + 1))
    P_micro_aggr = P_micro_aggr + ((TP_aggr + 1) / (TP_aggr + FN_aggr + 1))
end

P_macro = P_macro_aggr / #confusions
R_macro = R_macro_aggr / #confusions
P_micro = P_micro_aggr / #confusions
R_micro = R_micro_aggr / #confusions

print('P_macro: '..P_macro)
print('R_macro: '..R_macro)
print('F1_macro: '..((2 * P_macro * R_macro) / (P_macro + R_macro)))
print('P_micro: '..P_micro)
print('R_micro: '..R_micro)
print('F1_micro: '..((2 * P_micro * R_micro) / (P_micro + R_micro)))

In [16]:
-- 1 epochs
-- P_macro: 0.69945066586954
-- R_macro: 0.64201492791503
-- F1_macro: 0.66950322230995
-- P_micro: 0.68537999276845
-- R_micro: 0.59231310709299
-- F1_micro: 0.63545706414173

-- 3 epochs
-- P_macro: 0.63519394650611
-- R_macro: 0.79737983358683
-- F1_macro: 0.70710611962693
-- P_micro: 0.81134224534116
-- R_micro: 0.60977941734798
-- F1_micro: 0.6962666387025

-- 4 epochs
-- P_macro: 0.62607635104592
-- R_macro: 0.82234686224491
-- F1_macro: 0.71091365849986
-- P_micro: 0.83636380928908
-- R_micro: 0.60415084660385
-- F1_micro: 0.70154080194013

## Sources
* **(Kim, 2014):** Convolutional Neural Networks for Sentence Classification by Yoon Kim
* **(Zhang and Wallace, 2015):** A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification by Ye Zhang, Byron Wallace

In [9]:
criterion = nn.BCECriterion()
model = get_model(150, 300, .5, 3)
x_train, y_train, x_test, y_test = get_kth_partition(0)

for e = 1, epochs do
    print("training epoch "..e)
    -- train with batch_size = 100
    train(model, criterion, 100, x_train, y_train)
end


retrieving ../data/prepped_train_X_word2vec_wordvecs_0.h5	


training epoch 1	
training network	
getting 1 to 100 of 1930	


bad argument #2 to '?' (index out of bound at /Users/nexus/torch/pkg/torch/generic/Tensor.c:779)
stack traceback:
	[C]: at 0x1204e140
	[C]: in function '__newindex'
	[string "-- FIXME: replace this with something better..."]:6: in function 'copy_example'
	[string "function train(model, criterion, batch_size, ..."]:36: in function 'opfunc'
	/Users/nexus/torch/install/share/lua/5.1/optim/adadelta.lua:29: in function 'adadelta'
	[string "function train(model, criterion, batch_size, ..."]:54: in function 'train'
	[string "criterion = nn.BCECriterion()..."]:8: in main chunk
	[C]: in function 'xpcall'
	/Users/nexus/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/nexus/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/nexus/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/nexus/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/nexus/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Us

In [10]:
require 'csvigo'
X_csv = csvigo.load({path = "../data/her_movie_wordvecs.csv",
        verbose = false, mode = "raw"})

In [11]:
table.remove(X_csv, 1)

{
  1 : 
  2 : 0
  3 : 1
  4 : 2
  5 : 3
  6 : 4
  7 : 5
  8 : 6
  9 : 7
  10 : 8
  11 : 9
  12 : 10
  13 : 11
  14 : 12
  15 : 13
  16 : 14
  17 : 15
  18 : 16
  19 : 17
  20 : 18
  21 : 19
  22 : 20
  23 : 21
  24 : 22
  25 : 23
  26 : 24
  27 : 25
  28 : 26
  29 : 27
  30 : 28
  31 : 29
  32 : 30
  33 : 31
  34 : 32
  35 : 33
  36 : 34
  37 : 35
  38 : 36
  39 : 37
  40 : 38
  41 : 39
  42 : 40
  43 : 41
  44 : 42
  45 : 43
  46 : 44
  47 : 45
  48 : 46
  49 : 47
  50 : 48
  51 : 49
  52 : 50
  53 : 51
  54 : 52
  55 : 53
  56 : 54
  57 : 55
  58 : 56
  59 : 57
  60 : 58
  61 : 59
  62 : 60
  63 : 61
  64 : 62
  65 : 63
  66 : 64
  67 : 65
  68 : 66
  69 : 67
  70 : 68
  71 : 69
  72 : 70
  73 : 71
  74 : 72
  75 : 73
  76 : 74
  77 : 75
  78 : 76
  79 : 77
  80 : 78
  81 : 79
  82 : 80
  83 : 81
  84 : 82
  85 : 83
  86 : 84
  87 : 85
  88 : 86
  89 : 87
  90 : 88
  91 : 89
  92 : 90
  93 : 91
  94 : 92
  95 : 93
  96 : 94
  97 : 95
  98 : 96
  99 : 97
  100 : 98
  101 : 99
  102 :

In [23]:
print(#X_csv)

146	


In [24]:
print(#(X_csv[1]))

301	


In [37]:
x = copy_example(torch.Tensor(X_csv)[{{1,146}, {2, 301}}])

In [39]:
model:forward(x)

 0.5382  0.5065  0.5965  0.4645  0.4893  0.5579  0.4914  0.4320
[torch.DoubleTensor of size 1x8]

